In [1]:
pip install cohere --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install  pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import cohere

# 1. Carga de datos
df_mc = pd.read_csv("C:\\Users\\dairo\\Documents\\universidad\\Especializacion analitica estrategica de datos\\Analitica para la toma de decisiones\\Proyecto\\main\\Projecto_TRM-main\\Datos\\predicciones_bitcoinMontecarlo.csv", parse_dates=["Fecha"])
df_xgb    = pd.read_csv("C:\\Users\\dairo\\Documents\\universidad\\Especializacion analitica estrategica de datos\\Analitica para la toma de decisiones\\Proyecto\\main\\Projecto_TRM-main\\Datos\\predicciones_XGBoost.csv",         parse_dates=["Fecha"])

In [4]:
df_mc.head()

,Fecha,Prediccion,Cambio_7d,Cambio_1d
0,2025-04-07,79235.335938,-4.014078,1.305195
1,2025-04-08,76271.953125,-10.446525,-3.739976
2,2025-04-09,82573.953125,0.106979,8.262539
3,2025-04-10,79725.796875,-4.063678,-3.449219
4,2025-04-11,82945.039062,-1.071952,4.037893


In [5]:
df_xgb = df_xgb.rename(columns={"Predicción": "Prediccion"})
df_xgb.head()

,Fecha,Prediccion,Cambio_7d,Cambio_1d
0,2025-04-06,83955.670,1.968978,0.539939
1,2025-04-07,85497.270,3.571651,1.836209
2,2025-04-08,84268.820,-1.057133,-1.436833
3,2025-04-09,84663.805,2.640571,0.468719
4,2025-04-10,84550.580,1.742119,-0.133737


In [6]:
# 2. Función para obtener las predicciones de una fecha dada
def get_predictions_for_date(fecha_str):
    fecha = pd.to_datetime(fecha_str)
    # Filtrar cada dataset
    mc_row  = df_mc[df_mc["Fecha"] == fecha]
    xgb_row = df_xgb[df_xgb["Fecha"] == fecha]
    if mc_row.empty or xgb_row.empty:
        raise ValueError(f"No hay predicciones para la fecha {fecha_str}")
    return {
        "montecarlo": float(mc_row["Prediccion"].iloc[0]),
        "xgboost":    float(xgb_row["Prediccion"].iloc[0])
    }


In [9]:
# 3. Construcción del prompt para el LLM
def build_prompt(fecha_str, preds: dict):
    return f"""
Eres un analista financiero experto en criptomonedas.
Tienes dos predicciones para el precio de Bitcoin el día {fecha_str}:
  • Predicción Monte Carlo: USD {preds['montecarlo']:.2f}
  • Predicción XGBoost:    USD {preds['xgboost']:.2f}

Contexto adicional:
  • Precio actual de Bitcoin (spot): USD {{precio_spot_actual}}
  • Tu objetivo es maximizar rentabilidad y controlar riesgo.

Según estas predicciones, ¿qué recomendarías hacer ese día?
  - “Vender” si crees que el precio va a bajar,
  - “Comprar” si va a subir,
  - o “Mantener” si no hay una señal clara.
Explica brevemente tu razonamiento (1–2 frases).
"""

In [8]:
# 3b. Prompt con datos de cada horizonte y decisión
def build_prompt_multihorizonte(fecha_str: str,
                                precio_spot_actual: float,
                                pred_1d_mc: float,
                                pred_1d_xgb: float,
                                avg_7d_mc: float,
                                avg_7d_xgb: float,
                                avg_total_mc: float,
                                avg_total_xgb: float,
                                decision: str) -> str:
    return f"""
Eres un analista financiero experto en criptomonedas.

Fecha objetivo: {fecha_str}
Precio spot actual: USD {precio_spot_actual:.2f}

Predicción a 1 día:
  • Monte Carlo: USD {pred_1d_mc:.2f}
  • XGBoost:     USD {pred_1d_xgb:.2f}

Promedio predicciones 7+ días:
  • Monte Carlo: USD {avg_7d_mc:.2f}
  • XGBoost:     USD {avg_7d_xgb:.2f}

Promedio predicciones totales (desde hoy en adelante):
  • Monte Carlo: USD {avg_total_mc:.2f}
  • XGBoost:     USD {avg_total_xgb:.2f}

Según estas señales combinadas, **la recomendación automática es: {decision}**.

Ahora, explica brevemente (1–2 frases) por qué esa recomendación (“{decision}”).
"""

In [9]:
# 4. Llamada a la API de Cohere con lógica de recomendación por horizontes
def recomendar_para_fecha(fecha_str: str,
                         api_key: str,
                         df_mc: pd.DataFrame,
                         df_xgb: pd.DataFrame,
                         precio_spot_actual: float,
                         umbral_1d: float = 0.01,
                         umbral_7d: float = 0.02,
                         umbral_total: float = 0.015):
    """
    - df_mc, df_xgb: DataFrames con ['Fecha','Prediccion','Cambio_7d','Cambio_1d']
    - fecha_str: 'YYYY-MM-DD'
    - precio_spot_actual: float
    - umbral_1d: fracción para señal a 1 día (ej. 0.01 para 1%)
    - umbral_7d: fracción para señal a 7+ días (ej. 0.02 para 2%)
    - umbral_total: fracción para señal de todas las predicciones futuras
    """
    # Asegurar datetime
    for df in (df_mc, df_xgb):
        df['Fecha'] = pd.to_datetime(df['Fecha'])

    fecha = pd.to_datetime(fecha_str)
    fecha_1d = fecha + pd.Timedelta(days=1)
    fecha_7d = fecha + pd.Timedelta(days=7)

    # Filtrar horizontes
    mc_1d = df_mc[df_mc['Fecha'] == fecha_1d]['Prediccion']
    xgb_1d = df_xgb[df_xgb['Fecha'] == fecha_1d]['Prediccion']

    mc_7d_plus = df_mc[df_mc['Fecha'] >= fecha_7d]['Prediccion']
    xgb_7d_plus = df_xgb[df_xgb['Fecha'] >= fecha_7d]['Prediccion']

    mc_total = df_mc[df_mc['Fecha'] >= fecha]['Prediccion']
    xgb_total = df_xgb[df_xgb['Fecha'] >= fecha]['Prediccion']

    # Valores de referencia (si faltan, usamos NaN)
    pred_1d_mc = mc_1d.iloc[0] if not mc_1d.empty else float('nan')
    pred_1d_xgb = xgb_1d.iloc[0] if not xgb_1d.empty else float('nan')

    avg_7d_mc = mc_7d_plus.mean() if not mc_7d_plus.empty else float('nan')
    avg_7d_xgb = xgb_7d_plus.mean() if not xgb_7d_plus.empty else float('nan')

    avg_total_mc = mc_total.mean() if not mc_total.empty else float('nan')
    avg_total_xgb = xgb_total.mean() if not xgb_total.empty else float('nan')

    # Señales
    señal_1d = (pred_1d_mc > precio_spot_actual * (1 + umbral_1d)
                 and pred_1d_xgb > precio_spot_actual * (1 + umbral_1d))
    señal_7d = (avg_7d_mc > precio_spot_actual * (1 + umbral_7d)
                 and avg_7d_xgb > precio_spot_actual * (1 + umbral_7d))
    señal_total = (avg_total_mc > precio_spot_actual * (1 + umbral_total)
                   and avg_total_xgb > precio_spot_actual * (1 + umbral_total))

    # Decisión: si la mayoría de señales (1d,7d,total) apuntan igual
    votos = {'Comprar': 0, 'Vender': 0}
    for señal, umbral in [(señal_1d, umbral_1d),
                          (señal_7d, umbral_7d),
                          (señal_total, umbral_total)]:
        if señal:
            votos['Comprar'] += 1
        else:
            # inverso: si la predicción está por debajo del spot menos umbral
            votos['Vender'] += 1

    decision = 'Comprar' if votos['Comprar'] > votos['Vender'] else 'Vender'
    # En empate, mantenemos
    if votos['Comprar'] == votos['Vender']:
        decision = 'Mantener'

    # Construir prompt
    prompt = build_prompt_multihorizonte(
        fecha_str, precio_spot_actual,
        pred_1d_mc, pred_1d_xgb,
        avg_7d_mc, avg_7d_xgb,
        avg_total_mc, avg_total_xgb,
        decision
    )

    # Llamada a Cohere
    co = cohere.ClientV2(api_key)
    response = co.chat(
        model="command-a-03-2025",
        messages=[{"role": "user", "content": prompt.strip()}]
    )
    return response

In [10]:
pip install yfinance 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import yfinance as yf
import datetime

today = datetime.date.today()
# Nueva fecha restar un día
yesterday = today - datetime.timedelta(days=1)
print(today)
btc_data = yf.download("BTC-USD", start=yesterday, end=today, interval="1d")
btc_data.head()
if not btc_data.empty:
  #close_price = btc_data['Close'][0]
  print(btc_data.head())
  #tipo de dato btc_data
  print(type(btc_data))
  # Obtener el precio de cierre segun fecha yesterday
  close_price = btc_data.loc[yesterday.strftime("%Y-%m-%d")]['Close']
  print("Precio de cierre de BTC-USD: ", close_price)
  #print(close_price)
else:
  print("No data found for today.")

2025-04-13
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price              Close        High        Low          Open       Volume
Ticker           BTC-USD     BTC-USD    BTC-USD       BTC-USD      BTC-USD
Date                                                                      
2025-04-12  85287.109375  85856.1875  82769.375  83404.515625  24258059104
<class 'pandas.core.frame.DataFrame'>
Precio de cierre de BTC-USD:  Ticker
BTC-USD    85287.109375
Name: 2025-04-12 00:00:00, dtype: float64


In [12]:
print( close_price)

Ticker
BTC-USD    85287.109375
Name: 2025-04-12 00:00:00, dtype: float64


In [13]:
precio_float = close_price["BTC-USD"]
print(precio_float)  


85287.109375


In [14]:
# Ejemplo de uso:
if __name__ == "__main__":
    API_KEY = "1gyw72nkaPytUjvZw30ntTI5mqy3mwk1MNOzjO2x"
    fecha_consulta = "2025-04-13"
    
    respuesta = recomendar_para_fecha(fecha_consulta, API_KEY, df_mc, df_xgb,precio_spot_actual=precio_float)
    print(respuesta)


id='942f1202-229d-43c5-b120-9381ff104597' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='La recomendación de "Vender" se basa en que tanto las predicciones a corto plazo (1 día) como las proyecciones promedio a 7+ días y totales muestran una tendencia a la baja en el precio de la criptomoneda, con los modelos Monte Carlo y XGBoost coincidiendo en una disminución del valor actual.')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=216.0, output_tokens=70.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=711.0, output_tokens=72.0)) logprobs=None
